In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import os
import sys

FAST_MODE = True
SUBSET_SIZE = 14500  # Тільки 500 зображень для тренування!
TEST_SIZE = 1500    # Тільки 100 для тесту
BATCH_SIZE = 32
EPOCHS = 8         # Дуже мало епох!
LATENT_DIM = 16    # Дуже малий latent space

# --- Cell 1: АРХІТЕКТУРА ---
class FastAutoencoder(nn.Module):
    def __init__(self, latent_dim=16):
        super(FastAutoencoder, self).__init__()
        
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 8, kernel_size=3, stride=2, padding=1),  # 64x64 -> 32x32
            nn.ReLU(),
            nn.Conv2d(8, 16, kernel_size=3, stride=2, padding=1), # 32x32 -> 16x16
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(16 * 16 * 16, latent_dim)
        )
        
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 16 * 16 * 16),
            nn.ReLU(),
            nn.Unflatten(1, (16, 16, 16)),
            nn.ConvTranspose2d(16, 8, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(8, 3, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        z = self.encoder(x)
        return self.decoder(z)

# --- Cell 2: ЗАВАНТАЖЕННЯ (ВИПРАВЛЕНА ВЕРСІЯ) ---
def load_tiny_dataset(data_path, img_size=64):
    """Завантажуємо вибірку даних"""
    transform = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
    ])
    
    try:
        # Перевірка чи існує шлях
        if not os.path.exists(data_path):
            raise FileNotFoundError(f"Шлях {data_path} не існує")
            
        full_dataset = ImageFolder(root=data_path, transform=transform)
        print(f" Успішно завантажено: {len(full_dataset)} зображень")
        print(f"   Класи: {full_dataset.classes}")
        
        # Беремо лише невелику вибірку для швидкості
        total_samples = min(SUBSET_SIZE + TEST_SIZE, len(full_dataset))
        indices = torch.randperm(len(full_dataset))[:total_samples]
        
        train_size = SUBSET_SIZE
        train_indices = indices[:train_size]
        test_indices = indices[train_size:train_size + TEST_SIZE]
        
        train_dataset = Subset(full_dataset, train_indices)
        test_dataset = Subset(full_dataset, test_indices)
        
        print(f"📊 Використовуємо: {len(train_dataset)} тренувальних, {len(test_dataset)} тестових")
        
        return train_dataset, test_dataset, full_dataset.classes
        
    except Exception as e:
        print(f"Помилка завантаження: {e}")
        print("Створюємо демо-дані...")
        from torchvision.datasets import FakeData
        train_dataset = FakeData(size=SUBSET_SIZE, image_size=(3, 64, 64), num_classes=3, transform=transforms.ToTensor())
        test_dataset = FakeData(size=TEST_SIZE, image_size=(3, 64, 64), num_classes=3, transform=transforms.ToTensor())
        class_names = ['cat', 'dog', 'wild']
        
        print(f"📊 Демо-дані: {len(train_dataset)} тренувальних, {len(test_dataset)} тестових")
        
        return train_dataset, test_dataset, class_names

# Завантажуємо дані
DATA_PATH = "D:/Kotopes/Kotopes/data"
print(f" Завантаження даних з: {DATA_PATH}")
train_dataset, test_dataset, class_names = load_tiny_dataset(DATA_PATH)

# --- Cell 3: ТРЕНУВАННЯ ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f" Використовується пристрій: {device}")

model = FastAutoencoder(latent_dim=LATENT_DIM).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

def fast_train(model, train_loader, test_loader, epochs=EPOCHS):
    train_losses = []
    
    for epoch in range(epochs):
        # Тренування
        model.train()
        train_loss = 0
        for data, _ in train_loader:
            data = data.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, data)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        train_loss /= len(train_loader)
        train_losses.append(train_loss)
        print(f'Epoch {epoch+1}/{epochs}: Loss: {train_loss:.4f}')
    
    return train_losses

print("Початок швидкого тренування...")
train_losses = fast_train(model, train_loader, test_loader)

# --- Cell 4: ШВИДКИЙ АНАЛІЗ (ВИПРАВЛЕНА ВЕРСІЯ) ---
print("\n=== АНАЛІЗ ===")

# Визначаємо нормальний клас
NORMAL_CLASS = class_names.index('cat') if 'cat' in class_names else 0
print(f"Нормальний клас: {class_names[NORMAL_CLASS]}")

# Фільтруємо тестові дані (ВИПРАВЛЕНА ЧАСТИНА)
test_normal_indices = []
test_anomalous_indices = []

for i in range(len(test_dataset)):
    img, label = test_dataset[i]  # Виправлено - без зайвого unpacking
    if label == NORMAL_CLASS:
        test_normal_indices.append(i)
    else:
        test_anomalous_indices.append(i)

# Обмежуємо кількість для швидкості
test_normal_indices = test_normal_indices[:20]
test_anomalous_indices = test_anomalous_indices[:20]

test_normal = Subset(test_dataset, test_normal_indices)
test_anomalous = Subset(test_dataset, test_anomalous_indices)

print(f" Тестові дані: {len(test_normal)} нормальних, {len(test_anomalous)} аномальних")

# Функція для швидкого обчислення помилок
def fast_errors(model, dataset, device):
    model.eval()
    errors = []
    with torch.no_grad():
        loader = DataLoader(dataset, batch_size=32)
        for images, _ in loader:
            images = images.to(device)
            reconstructed = model(images)
            error = torch.mean((images - reconstructed) ** 2, dim=[1,2,3])
            errors.extend(error.cpu().numpy())
    return np.array(errors)

normal_errors = fast_errors(model, test_normal, device)
anomalous_errors = fast_errors(model, test_anomalous, device)

threshold = np.percentile(normal_errors, 95)
detection_rate = np.mean(anomalous_errors > threshold) * 100
false_positive = np.mean(normal_errors > threshold) * 100

print(f"РЕЗУЛЬТАТИ:")
print(f"   - Threshold: {threshold:.4f}")
print(f"   - Виявлено аномалій: {detection_rate:.1f}%")
print(f"   - Помилка (FPR): {false_positive:.1f}%")

# --- Cell 5: ВІЗУАЛІЗАЦІЯ ---
plt.figure(figsize=(12, 3))

# Графік тренування
plt.subplot(1, 3, 1)
plt.plot(train_losses)
plt.title('Втрати при тренуванні')
plt.xlabel('Епоха')
plt.grid(True, alpha=0.3)

# Розподіл помилок
plt.subplot(1, 3, 2)
plt.hist(normal_errors, alpha=0.7, label='Нормальні', bins=15, color='green')
plt.hist(anomalous_errors, alpha=0.7, label='Аномальні', bins=15, color='red')
plt.axvline(threshold, color='black', linestyle='--', label=f'Threshold: {threshold:.3f}')
plt.legend()
plt.title('Розподіл помилок')
plt.grid(True, alpha=0.3)

# Приклади
plt.subplot(1, 3, 3)
# Беремо один приклад для демонстрації
sample_idx = 0
sample_img, sample_label = test_dataset[sample_idx]  # Виправлено
sample_img = sample_img.unsqueeze(0).to(device)
with torch.no_grad():
    reconstructed = model(sample_img)

plt.imshow(sample_img[0].cpu().permute(1, 2, 0))
plt.title(f'Приклад: {class_names[sample_label]}')
plt.axis('off')

plt.tight_layout()
plt.show()

# --- Cell 6: ЛАТЕНТНИЙ ПРОСТІР ---
print("\n=== ЛАТЕНТНИЙ ПРОСТІР ===")

def fast_latent_vectors(model, dataset, device, max_samples=50):
    model.eval()
    latent_vectors = []
    labels = []
    
    with torch.no_grad():
        loader = DataLoader(dataset, batch_size=32)
        for images, batch_labels in loader:
            images = images.to(device)
            latent = model.encoder(images)
            latent_vectors.extend(latent.cpu().numpy())
            labels.extend(batch_labels.numpy())
            
            if len(latent_vectors) >= max_samples:
                break
    
    return np.array(latent_vectors)[:max_samples], np.array(labels)[:max_samples]

latent_vectors, labels = fast_latent_vectors(model, test_dataset, device)

print(f" Витягнуто {len(latent_vectors)} латентних векторів")

# t-SNE для латентного простору
if len(latent_vectors) > 10:
    print("🔍 Застосовуємо t-SNE...")
    tsne = TSNE(n_components=2, random_state=42, perplexity=min(10, len(latent_vectors)-1))
    latent_2d = tsne.fit_transform(latent_vectors)
    
    plt.figure(figsize=(8, 4))
    
    # За класами
    plt.subplot(1, 2, 1)
    colors = ['red', 'blue', 'green', 'orange', 'purple']
    for class_idx in range(len(class_names)):
        mask = labels == class_idx
        if np.sum(mask) > 0:
            plt.scatter(latent_2d[mask, 0], latent_2d[mask, 1], 
                       label=class_names[class_idx], alpha=0.7, s=30)
    plt.legend()
    plt.title('Латентний простір за класами')
    plt.grid(True, alpha=0.3)
    
    # Нормальні vs аномальні
    plt.subplot(1, 2, 2)
    is_normal = (labels == NORMAL_CLASS).astype(int)
    plt.scatter(latent_2d[is_normal==1, 0], latent_2d[is_normal==1, 1], 
               c='green', label=f'Нормальні ({class_names[NORMAL_CLASS]})', alpha=0.7, s=30)
    plt.scatter(latent_2d[is_normal==0, 0], latent_2d[is_normal==0, 1], 
               c='red', label='Аномальні', alpha=0.7, s=30)
    plt.legend()
    plt.title('Поділ нормальних/аномальних')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print(" Недостатньо даних для t-SNE")

print("\nЕКСПЕРИМЕНТ ЗАВЕРШЕНО!")
print(f" Використано: {SUBSET_SIZE} тренувальних та {TEST_SIZE} тестових зображень")
print(f" Кількість епох: {EPOCHS}")
print(f" Класи: {class_names}")

 Завантаження даних з: D:/Kotopes/Kotopes/data
 Успішно завантажено: 16130 зображень
   Класи: ['train', 'val']
📊 Використовуємо: 14500 тренувальних, 1500 тестових
 Використовується пристрій: cpu
Початок швидкого тренування...
Epoch 1/8: Loss: 0.0341
